In [273]:
%pip install scikit-learn


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [274]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
sys.path.append('../')
from utils import remove_similar_rows_per_player, find_lines_with_player, del_patterns


In [275]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [276]:
# Filter out the German data and reindex
df_ger = df[df["language"] == "de"]
df_ger = df_ger.reset_index(drop=True)

In [277]:
# Remove the similiar rows (The Function is imported from utils on top)
df = remove_similar_rows_per_player(df_ger, df_ger['player'].unique())


In [278]:
# transform data into lower case
data_lower = df.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()
data_lower.head()

data_wo_pattern = data_lower.copy()

# delete content patterns
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))

In [280]:
pattern = ['nutze kicker', 'mit dem', 'nur €2,49', 'bereits pur-abonnent?', 'alle antworten', 'hinweis zur verarbeitung', 'werbung','olympia-verlags', 'bild', 'g+j medien gmbh', 'services',]

In [281]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: del_patterns_1(str(x), pattern))                                                      

In [282]:
data_wo_pattern['data'].iloc[5]

'spiele & ergebnisse der europa league\\nachtelfinale der europa league\\n            \\n: \\n\\n\\nfc arsenal strauchelt in lissabon\\njuventus turin gewinnt knapp\\n            \\n: \\n\\n\\npassable ausgangslage für den sc freiburg\\n3:3 gegen saint-gilloise\\n            \\n: \\n\\n\\nunion berlin gibt sich nicht auf\\nauslosung zur europa league\\n            \\n: \\n\\n\\nsc freiburg zieht mit juventus das große los\\ndie ungarn tauchten erst in der 26. minute vor dem leverkusener tor auf. das aber sehr gefährlich. kristoffer zachariassen lupfte den ball aus sieben metern über den herausstürmenden bayer-torhüter lukas hradecky – aber an die latte. dieses ausrufezeichen gab budapest rückenwind, bis zur pause hielt der überlegene tabellenführer der ungarischen liga das spiel offen.\\nund auch danach agierten beide teams weitgehend auf augenhöhe, ehe kurz vor schluss bayer doch noch das so wichtige zweite tor gelang. der eingewechselte adam hlouzek setzte zunächst einen freistoß aus

## Find sentences with Playername

In [283]:
data_with_playernames = find_lines_with_player(df, df['player'].unique())

In [284]:
data_with_playernames

,data,player,language,publishedAt
0,{'content': 'Trainer Xabi Alonso vor den mitge...,Exequiel Palacios,de,2023-02-24T09:33:31Z
1,{'content': 'Nutze kicker auf seinen digitalen...,Exequiel Palacios,de,2023-03-03T21:35:13Z
2,{'content': 'Nutze kicker auf seinen digitalen...,Exequiel Palacios,de,2023-03-07T11:34:39Z
3,{'content': 'Mitchel Bakker am Mittwoch im Abs...,Exequiel Palacios,de,2023-03-08T14:25:18Z
4,{'content': '© G+J Medien GmbH,Exequiel Palacios,de,2023-03-09T19:36:53Z
...,...,...,...,...
21,"{'content': ""Der SC Freiburg und Bayer Leverku...",Piero Hincapié,de,2023-02-26T16:28:57Z
22,{'content': 'Ein Monaco-Betreuer packt Sardar ...,Piero Hincapié,de,2023-02-22T16:59:33Z
23,{'content': 'Leverkusens Torwart Lukas Hradeck...,Piero Hincapié,de,2023-02-17T17:14:46Z
24,{'content': 'Das WichtigsteTut es in der Kolle...,Piero Hincapié,de,2023-02-17T08:56:42Z


In [285]:
data_with_playernames = find_lines_with_player(data_wo_pattern, data_wo_pattern['player'].unique(), n_lines=5)

In [286]:
data_with_playernames['data'].iloc[0]

'trainer xabi alonso vor den mitgereisten fans in monacoglücksmoment! exequiel palacios hebt nach seinem tor zum 2:1 abnach dem euro-highlight geht es in der bundesliga am sonntag (15.30 uhr) beim sc freiburg weiter.„wir kennen freiburg. sie haben gerade einen sehr guten moment, sie spielen mit einer hohen intensität, ähnlich wie mainz. vielleicht war das spiel gegen mainz eine gute schule für uns für das freiburg-spiel“, sagt alonso.der bayer-tross bleibt noch einen tag in monaco, samstag um 12 uhr fliegt das team dann nach basel und trainiert anschließend in freiburg.wegen leverkusens strafstoß-krise? freche elfer-aktion von mainz-torwart!'

In [287]:
data_rm = data_with_playernames.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm['data'] = data_rm['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm['data'] = data_rm['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm['data'] = data_rm['data'].apply(strip_multiple_whitespaces)

In [288]:
data_rm['data'].iloc[6]

'mit einem späten tor hat joker sven michel dem fc union berlin in einem mitreißenden achtelfinal hinspiel der europa league ein unentschieden gerettet und die chancen auf das weiterkommen deutlich verbessert die berliner taten sich gegen royale union saint gilloise im heimischen stadion an der alten försterei erneut schwer und kamen am ende zu einem vor zuschauern schossen victor boniface und minute und yorbe vertessen die gäste dreimal in führung josip juranovic robin knoche und michel erzielten die jeweiligen ausgleichstore den fußball bundesligisten der comeback qualitäten bewies auf dem platz gab aber zunächst bayer den ton an dass demirbay das frühe führungstor mit einem schönen rechtsschuss aus metern erzielte konnte man mit dem attribut „ausgerechnet“ versehen denn der ex nationalspieler ist unter alonso kaum gefragt und bei der aufstellung mit ihm und amiri als doppel sechs hatte so manchen bayer fan eine unschöne erinnerung ereilt ein einziges mal hatten die beiden ex hoffenh

## Stopwords

In [289]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts']

stop_words_to_add = ['vor', 'in', 'den', 'dem', 'beim', 'wir', 'der', 'ist','ende', 'seite', 'ersten', 'fürs', 'eh', 'blick', 'schon', 'zumal', 'erst', 'mal', 'bild', 't']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

german_stop_words.append(stop_words_to_add)

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=german_stop_words)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm['data'] = data_rm['data'].apply(remove_stopwords_from_text)


## Pattern Regex


In [290]:
'''patternlist = [
    r'^\s*nutze kicker.*?\.',
    r'^\s*die zustimmung.*?\.',
]

for pattern in patternlist:

    data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))
'''


'patternlist = [\n    r\'^\\s*nutze kicker.*?\\.\',\n    r\'^\\s*die zustimmung.*?\\.\',\n]\n\nfor pattern in patternlist:\n\n    data_wo_pattern[\'data\'] = data_wo_pattern[\'data\'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))\n'